In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

In [9]:
def extract_features(image):
#     surf = cv2.xfeatures2d.SURF_create()   
#     kp, des = surf.detectAndCompute(image, None)
    sift = cv2.xfeatures2d.SIFT_create()
    kp, des = sift.detectAndCompute(image, None)
    return kp, des


def match_features(des1, des2):
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)  
    search_params = dict(checks=50)  
    
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    match = flann.knnMatch(des1, des2, k=2) 
    return match


def filter_matches(match, dist_threshold=0.75):
    filtered_match = []
    for (m, n) in match:
        if m.distance < dist_threshold * n.distance:
            filtered_match.append(m)
    return filtered_match


def estimate_homography(match, kp1, kp2):
    src_pts = []
    dst_pts = []    
    for m in match:  
        train_idx = m.trainIdx   
        query_idx = m.queryIdx
    
        p1_x, p1_y = kp1[query_idx].pt
        src_pts.append([p1_x, p1_y]) 

        p2_x, p2_y = kp2[train_idx].pt   
        dst_pts.append([p2_x, p2_y])
        
    homography, mask = cv2.findHomography(np.array(src_pts), np.array(dst_pts), cv2.RANSAC, 5.0) 
    return homography


def main(ref_image, template, video):
    ref_image = cv2.imread(ref_image)  ## load gray if you need.
    template = cv2.imread(template)  ## load gray if you need.
    video = cv2.VideoCapture(video)
    ref_image = cv2.resize(ref_image, template.shape[:2])
    
    film_h, film_w = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT)), int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    film_fps = video.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
    videowriter = cv2.VideoWriter("ar_video_2.mp4", fourcc, film_fps, (film_w, film_h))
    
    i = 0
    kp_temp, des_temp = extract_features(template)
    while(video.isOpened()):
        ret, frame = video.read()
        print('Processing frame {}'.format(i))
        if ret:  ## check whethere the frame is legal, i.e., there still exists a frame
            
            ## TODO: homography transform, feature detection, ransanc, etc.
            kp_frame, des_frame = extract_features(frame)
            # display = cv2.drawKeypoints(frame, kp_frame, None)
            match = match_features(des_temp, des_frame)
            filtered_match = filter_matches(match)
            H = estimate_homography(filtered_match, kp_temp, kp_frame)
            
            h, w, c = ref_image.shape
            for y in range(h):
                for x in range(w):
                    pixel = ref_image[y, x, :]
                    u = np.array([x, y, 1])
                    v = np.dot(H, u.T)
                    t_x = int(round(v[0] / v[2]))
                    t_y = int(round(v[1] / v[2]))
                    if t_x < frame.shape[1] and t_x >= 0 and t_y < frame.shape[0] and t_y >= 0:
                        frame[t_y-3 : t_y+3, t_x-3 : t_x+3, :] = pixel
                    else:
                        continue
            
            videowriter.write(frame)
            i += 1
            
        else:
            break
            
    video.release()
    videowriter.release()
    cv2.destroyAllWindows()
    

In [ ]:
if __name__ == '__main__':
    ## you should not change this part
    ref_path = './input/me.jpg'
    template_path = './input/marker.png'
    video_path = './input/ar_marker.mp4'
    main(ref_path,template_path,video_path)